Building a declination table
============================

In [1]:
from kanon.units import Sexagesimal
import astropy.table
from kanon.tables import HTable
import math

In [2]:
# For these calculations we will round after every operations.

from kanon.units.precision import get_context, TruncatureMode
get_context().mutate(tmode=TruncatureMode.ROUND)


In [3]:
OBLIQUITY = "23;51,20"

In [4]:
# We construct a table of sine from 0 to 90.

sin_table = HTable([
    list(Sexagesimal.range(91)),
    [round(Sexagesimal.from_float(math.sin(x * math.pi / 180), 3)) for x in range(91)]
], names=("Arg", "Val"), index="Arg")

sin_table

Arg,Val
object,object
00 ;,"00 ; 00,00,00"
01 ;,"00 ; 01,02,50"
02 ;,"00 ; 02,05,38"
03 ;,"00 ; 03,08,25"
04 ;,"00 ; 04,11,07"
05 ;,"00 ; 05,13,46"
06 ;,"00 ; 06,16,18"
07 ;,"00 ; 07,18,44"
08 ;,"00 ; 08,21,01"


In [5]:
# We apply a multiplication on all the sine values to get obl_table = Sin(x) * obl

obl = sin_table.get(Sexagesimal(OBLIQUITY))

obl_table = sin_table.apply("Val", lambda x: x * obl)
obl_table


Arg,Val
object,object
00 ;,"00 ; 00,00,00"
01 ;,"00 ; 00,25,25"
02 ;,"00 ; 00,50,49"
03 ;,"00 ; 01,16,12"
04 ;,"00 ; 01,41,33"
05 ;,"00 ; 02,06,54"
06 ;,"00 ; 02,32,11"
07 ;,"00 ; 02,57,26"
08 ;,"00 ; 03,22,37"


In [6]:
# Arcsin is sin_table taken from Val to Arg instead of Arg to Val.

arcsin_table = sin_table.copy(set_index="Val")

# For example, if we want arcsin(0.3) :

arcsin_table.get(0.3)


17 ; 27,29,49,59

In [7]:
# Finally, we apply arcsin_table.get on every obl_table values to find decl_table = arcsin(obltable)

decl_table = obl_table.apply("Val", arcsin_table.get)
decl_table

Arg,Val
object,object
00 ;,00 ;
01 ;,"00 ; 24,16,14"
02 ;,"00 ; 48,31,31"
03 ;,"01 ; 12,46,15"
04 ;,"01 ; 36,59,26"
05 ;,"02 ; 01,12,38"
06 ;,"02 ; 25,22,23"
07 ;,"02 ; 49,30,13"
08 ;,"03 ; 13,35,19"


In [8]:
# Let's make the same calculation with an interpolated sine table.

# First, extract every 12th row until the 60th then every 6th.

sin_table_grid = sin_table[
    [i for i in range(91) if i <= 60 and i % 12 == 0 or i > 60 and i % 6 == 0]
]

sin_table_grid

Arg,Val
object,object
00 ;,"00 ; 00,00,00"
12 ;,"00 ; 12,28,29"
24 ;,"00 ; 24,24,15"
36 ;,"00 ; 35,16,02"
48 ;,"00 ; 44,35,19"
"01,00 ;","00 ; 51,57,41"
"01,06 ;","00 ; 54,48,46"
"01,12 ;","00 ; 57,03,48"
"01,18 ;","00 ; 58,41,20"


In [9]:
# We can populate it with its interpolation method.

sin_table_pop = sin_table_grid.populate(list(Sexagesimal.range(91)), method="interpolate")

sin_table_pop

Arg,Val
object,object
00 ;,"00 ; 00,00,00"
01 ;,"00 ; 01,02,22"
02 ;,"00 ; 02,04,45"
03 ;,"00 ; 03,07,07"
04 ;,"00 ; 04,09,30"
05 ;,"00 ; 05,11,52"
06 ;,"00 ; 06,14,15"
07 ;,"00 ; 07,16,37"
08 ;,"00 ; 08,18,59"


In [10]:
# Comparing the 2nd difference between the true sine table and the interpolated one.

true_2diff = sin_table.diff(n=2, new_name="True Sine 2-diff")
interpolated_2diff = sin_table_pop.diff(n=2, new_name="Interpolated Sine 2-diff")

# Now we have 2 tables with the same arguments. We can compare them side to side by joining them.

astropy.table.join(true_2diff, interpolated_2diff)

Arg,True Sine 2-diff,Interpolated Sine 2-diff
object,object,object
00 ;,"00 ; 00,00,00","00 ; 00,00,00"
01 ;,"00 ; 01,02,50","00 ; 01,02,22"
02 ;,"-00 ; 00,00,02","00 ; 00,00,01"
03 ;,"-00 ; 00,00,01","-00 ; 00,00,01"
04 ;,"-00 ; 00,00,05","00 ; 00,00,01"
05 ;,"-00 ; 00,00,03","-00 ; 00,00,01"
06 ;,"-00 ; 00,00,07","00 ; 00,00,01"
07 ;,"-00 ; 00,00,06","-00 ; 00,00,01"
08 ;,"-00 ; 00,00,09","00 ; 00,00,00"


In [11]:
# With this newly built table we are able to construct a new declination table.

obl_interp = sin_table_pop.get(Sexagesimal(OBLIQUITY))

obl_table_interp = sin_table_pop.apply("Val", lambda x: x * obl)
arcsin_table_interp = sin_table_pop.copy(set_index="Val")

obl_table_interp.apply("Val", arcsin_table_interp.get)

Arg,Val
object,object
00 ;,00 ;
01 ;,"00 ; 24,15,35"
02 ;,"00 ; 48,32,08"
03 ;,"01 ; 12,47,31"
04 ;,"01 ; 37,03,40"
05 ;,"02 ; 01,19,51"
06 ;,"02 ; 25,35,26"
07 ;,"02 ; 49,51,59"
08 ;,"03 ; 14,07,21"
